In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
import itertools
import sklearn.feature_selection
import sklearn.model_selection as ms
import pickle
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import datetime
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from scipy.stats import skew
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from scipy import stats
from sklearn.preprocessing import normalize
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
plt.style.use('ggplot')
pd.options.display.max_columns = None
pd.options.display.max_rows = 6

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# Adding 'type' columns that contains 'train' or 'test', and add empty SalePrice columns to the test part
train.insert(0, 'type', 'train')
test.insert(0,'type','test')
test['SalePrice'] = 0.0

# merging the two dataframes
merged = pd.concat([train,test],axis=0).reset_index(drop=True)
merged['SalePrice'] = merged['SalePrice'].astype('float')

# Data cleaning
import Dprocessv3
merged = Dprocessv3.dataProcess(merged)

In [4]:
# numerical and categorical features + EXCLUDED FIRST
excl_features = merged[['Id','type','SalePrice']].columns
cat_features = merged.select_dtypes(include = ["object"]).columns.drop('type')
num_features = merged.select_dtypes(exclude = ["object"]).columns.drop(['Id','SalePrice'])

mergedNum = merged[num_features]
mergedCat = merged[cat_features]
mergedExc = merged[excl_features]

mergedNAtotal = merged.isnull().sum().sort_values(ascending=False)

print("NAs for numerical features in merged : " + str(mergedNum.isnull().values.sum()))

if mergedNum.isnull().values.sum() != 0:
    mergedNum = mergedNum.fillna(mergedNum.median())
    print("Remaining NAs for numerical features in merged : " + str(mergedNum.isnull().values.sum()))

NAs for numerical features in merged : 0


In [5]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in merged.columns:
    if merged[i].dtype in numeric_dtypes: 
        numerics2.append(i)

skew_features = merged[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)
skews = pd.DataFrame({'skew':skew_features})

In [6]:
high_skew = skew_features[skew_features > 0.5]
high_skew = high_skew
skew_index = high_skew.index

for i in skew_index:
    if i != 'SalePrice':
        merged[i]= boxcox1p(merged[i], boxcox_normmax(merged[i]+1))

        
skew_features2 = merged[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)
skews2 = pd.DataFrame({'skew':skew_features2})

/Users/macpro/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [7]:
# Create dummy features for categorical values via one-hot encoding

print("Remaining NAs for categorical features in merged : " + str(mergedCat.isnull().values.sum()))

print("NAs for categorical features in merged : " + str(mergedCat.isnull().values.sum()))

mergedCat = pd.get_dummies(mergedCat)


Remaining NAs for categorical features in merged : 0
NAs for categorical features in merged : 0


In [8]:
# rejoin excluded, categorical and numerical features
merged = pd.concat([mergedExc,mergedNum, mergedCat], axis = 1)

print("New number of features : " + str(merged.shape[1]))

New number of features : 305


In [9]:
merged.shape

(2916, 305)

In [10]:
train = merged[merged['type'] == 'train'].copy()
train.drop(columns=['Id','type'],inplace=True)
test = merged[merged['type'] == 'test'].copy()
test.drop(columns=['type','SalePrice'],inplace=True)

In [11]:
# taking the log of the saleprice

train['SalePrice'] = np.log(train['SalePrice'])
train['SalePrice'].describe()

count    1457.000000
mean       12.023714
std         0.399695
            ...     
50%        12.001505
75%        12.273731
max        13.534473
Name: SalePrice, Length: 8, dtype: float64

## Defining the train/test split (80/20)

In [12]:
house_features = train.drop('SalePrice',axis = 1)
prices = train['SalePrice']

x_train, x_test, y_train, y_test = ms.train_test_split(house_features, prices, 
                                                       test_size=0.2, random_state=111)

## Ridge

In [13]:
# 981 
# 0.12077
# R2: 94.20%
# trainRMSE: 9.69%
# testRMSE: 10.18%

ridcv = RidgeCV(alphas=np.linspace(1e-5,30,10000), normalize=False, scoring=None)

ridcv.fit(x_train,y_train)

trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=ridcv.predict(x_train)))
testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=ridcv.predict(x_test)))

print('The best alpha is: {:.6}\n\nThe ridge intercept is: {:.2f}\n\n{}\n\nThe model score is {:.2%}\n\nTrain RMSE: {:.2%}\n\nTest RMSE: {:.2%}\n\nDiff in RMSE: {:.0%}'.format(
    ridcv.alpha_,ridcv.intercept_, pd.Series(ridcv.coef_, index=x_train.columns), ridcv.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(ridcv.predict(test.drop(columns='Id')))}
#             ).to_csv('ridcv {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

The best alpha is: 19.784

The ridge intercept is: 12.07

LotFrontage              0.000637
LotArea                  0.000002
Street                  -0.023331
                           ...   
SaleCondition_Family    -0.024190
SaleCondition_Normal     0.030855
SaleCondition_Partial    0.019932
Length: 302, dtype: float64

The model score is 93.76%

Train RMSE: 10.05%

Test RMSE: 10.35%

Diff in RMSE: 0%


In [14]:
#  
# 
# R2: 
# trainRMSE: 
# testRMSE: 

# ridge = Ridge()
# parameters = {'alpha':np.linspace(20,25,1000)}
# ridgegs = ms.GridSearchCV(ridge, parameters, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
# ridgegs.fit(x_train, y_train)

# trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=ridgegs.predict(x_train)))
# testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=ridgegs.predict(x_test)))

# ridgegs.best_params_


# print('The best parameters are: {}\n\nThe ridge intercept is: {:.2f}\n\n{}\n\nThe model score is {:.2%}\n\nTrain RMSE: {:.2%}\n\nTest RMSE: {:.2%}\n\nDiff in RMSE: {:.0%}\n\n'.format(
#     ridgegs.alpha_,ridgegs.intercept_, pd.Series(ridgegs.coef_, index=x_train.columns), ridgegs.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

In [15]:
# ridcv = RidgeCV(alphas=[20.865865865865867], normalize=False, scoring=None)

# ridcv.fit(x_train,y_train)

# trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=ridcv.predict(x_train)))
# testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=ridcv.predict(x_test)))

# print('The best alpha is: {:.6}\n\nThe ridge intercept is: {:.2f}\n\n{}\n\nThe model score is {:.2%}\n\nTrain RMSE: {:.2%}\n\nTest RMSE: {:.2%}\n\nDiff in RMSE: {:.0%}'.format(
#     ridcv.alpha_,ridcv.intercept_, pd.Series(ridcv.coef_, index=x_train.columns), ridcv.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(ridcv.predict(test.drop(columns='Id')))}
#             ).to_csv('ridcv {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

## Elastic-net

In [16]:
# 917
# 0.11994
# R2: 0.9359
# trainRMSE: 0.101935
# testRMSE: 0.100320
# diff: -0.002

enet = ElasticNetCV(l1_ratio=np.linspace(0.01,1,100), alphas=np.linspace(1e-5,20,1000), max_iter=1000, normalize=False, cv=10, n_jobs=-1)

enet.fit(x_train, y_train)

trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=enet.predict(x_train)))
testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=enet.predict(x_test)))

print('The best alpha is: {:.6}\n\nThe elastic-net intercept is: {:.2f}\n\n{}\n\nThe model score is {:.2%}\n\nTrain RMSE: {:.2%}\n\nTest RMSE: {:.2%}\n\nDiff in RMSE: {:.0%}'.format(
    enet.alpha_,enet.intercept_, pd.Series(enet.coef_, index=x_train.columns), enet.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(enet.predict(test.drop(columns='Id')))}
#             ).to_csv('enet {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

...

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

The best alpha is: 0.02003

The elastic-net intercept is: 7.00

LotFrontage              0.000674
LotArea                  0.000002
Street                  -0.007077
                           ...   
SaleCondition_Family    -0.015482
SaleCondition_Normal     0.025867
SaleCondition_Partial    0.020944
Length: 302, dtype: float64

The model score is 93.14%

Train RMSE: 10.54%

Test RMSE: 10.25%

Diff in RMSE: -0%


/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [17]:
# 917
# 0.11994
# R2: 0.9359
# trainRMSE: 0.101935
# testRMSE: 0.100320
# diff: -0.002

# elasticnet = ElasticNet(max_iter=1000, normalize=False)
# parameters = {'alpha':np.linspace(1e-5,20,1000), 'l1_ratio':np.linspace(0.01,1,100)}

# elastgs = ms.GridSearchCV(elasticnet, parameters, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
# elastgs.fit(x_train, y_train)

# trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=elastgs.predict(x_train)))
# testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=elastgs.predict(x_test)))

# elastgs.best_params_

# pickle.dump(elastgs, open('elastgs', 'wb'))

# pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(enet.predict(test.drop(columns='Id')))}
#             ).to_csv('enet {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

In [18]:
# 
# 
# R2: 
# trainRMSE: 
# testRMSE: 
# diff: 

# parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
# svc = svm.SVC(gamma="scale")
# clf = GridSearchCV(svc, parameters, cv=5)
# clf.fit(iris.data, iris.target)

# enet = ElasticNetCV(l1_ratio=np.linspace(0.01,1,100), alphas=np.linspace(1e-5,20,1000), max_iter=1000, normalize=False, cv=10, n_jobs=-1)

# enet.fit(x_train, y_train)

# trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=enet.predict(x_train)))
# testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=enet.predict(x_test)))

# print('The best alpha is: {:.6}\n\nThe elastic-net intercept is: {:.2f}\n\n{}\n\nThe model score is {:.2%}\n\nTrain RMSE: {:.2%}\n\nTest RMSE: {:.2%}\n\nDiff in RMSE: {:.0%}'.format(
#     enet.alpha_,enet.intercept_, pd.Series(enet.coef_, index=x_train.columns), enet.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(enet.predict(test.drop(columns='Id')))}
#             ).to_csv('enet {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

## Lasso

In [19]:
#  alpha           UJ range              np.linspace(1e-4,1e-2,1000)
#  CV          100         10          100
# R2:        0.9366      0.9399      0.9374
# trainRMSE: 0.101363    0.098698    0.100718
# testRMSE:  0.100012    0.100196    0.099977
# diff:b       0.00       0.00         0.00

lassocv = LassoCV(alphas=np.linspace(1e-4,1e-3,1000), normalize=False,
                      max_iter=10000, n_jobs=-1, cv=100)

lassocv.fit(x_train, y_train)

trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=lassocv.predict(x_train)))
testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=lassocv.predict(x_test)))

print('The best alpha is: {:.6}\n\nThe lasso intercept is: {:.2f}\n\n{}\n\nThe model score is {:.2%}\n\nTrain RMSE: {:.2%}\n\nTest RMSE: {:.2%}\n\nDiff in RMSE: {:.0%}'.format(
    lassocv.alpha_,lassocv.intercept_, pd.Series(lassocv.coef_, index=x_train.columns), lassocv.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# pd.DataFrame({'Id':test['Id'],'SalePrice': np.exp(lassocv.predict(test.drop(columns='Id')))}
#             ).to_csv('lasso {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

...

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

The best alpha is: 0.00052973

The lasso intercept is: 6.66

LotFrontage              0.000689
LotArea                  0.000002
Street                  -0.000000
                           ...   
SaleCondition_Family    -0.041691
SaleCondition_Normal     0.012451
SaleCondition_Partial    0.000000
Length: 302, dtype: float64

The model score is 93.78%

Train RMSE: 10.04%

Test RMSE: 10.17%

Diff in RMSE: 0%


/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


## Random Forest

In [20]:
rf = RandomForestRegressor(n_estimators=1000,max_depth=10,max_features=0.5, min_samples_split=2, min_samples_leaf=1,n_jobs=-1, random_state=111)
rf.fit(x_train, y_train)

scores = ms.cross_val_score(rf, x_train, y_train, cv=10)

print(scores.mean())

trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=rf.predict(x_train)))
testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=rf.predict(x_test)))

print('\nThe model score is {:.2%}\n\nTrain RMSE: {:.2%}\n\nTest RMSE: {:.2%}\n\nDiff in RMSE: {:.0%}'.format(
    rf.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# pd.DataFrame({'Id':test['Id'],'SalePrice': np.exp(rf.predict(test.drop(columns='Id')))}
#             ).to_csv('rf {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

# 1000 trees
# 0.8395225412896815
# R2: 0.9517
# Train RMSE: 0.088419
# Test RMSE: 0.129310
# Diff in RMSE: 0.041

0.8832334909932751

The model score is 97.88%

Train RMSE: 5.86%

Test RMSE: 11.61%

Diff in RMSE: 6%


### Looping over max tree depth

In [21]:
# for i in range(10,20):
#     rf = RandomForestRegressor(n_estimators=500,max_depth=i,max_features='auto', min_samples_split=2, min_samples_leaf=1,n_jobs=-1)
#     rf.fit(x_train, y_train)

#     scores = ms.cross_val_score(rf, x_train, y_train, cv=10)
#     print('trial with depth = {}'.format(i))
#     print(scores.mean())

#     trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=rf.predict(x_train)))
#     testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=rf.predict(x_test)))

#     print('\nThe model score is {:.2%}\n\nTrain RMSE: {:.2%}\n\nTest RMSE: {:.2%}\n\nDiff in RMSE: {:.0%}'.format(
#         rf.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))



### Looping over percentage of features for each tree

In [22]:
# for i in np.linspace(0.05,1.0,20):
#     rf = RandomForestRegressor(n_estimators=500,max_depth=10,max_features=i, min_samples_split=2, min_samples_leaf=1,n_jobs=-1)
#     rf.fit(x_train, y_train)

#     scores = ms.cross_val_score(rf, x_train, y_train, cv=10)
#     print('trial with depth = {}'.format(i))
#     print(scores.mean())

#     trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=rf.predict(x_train)))
#     testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=rf.predict(x_test)))

#     print('\nThe model score is {:.2%}\n\nTrain RMSE: {:.2%}\n\nTest RMSE: {:.2%}\n\nDiff in RMSE: {:.2%}'.format(
#         rf.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))


## Gradient Boosting

In [23]:
gbst = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =111)

gbst.fit(x_train, y_train)

trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=gbst.predict(x_train)))
testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=gbst.predict(x_test)))

print('\nThe model score is {:.2%}\n\nTrain RMSE: {:.2%}\n\nTest RMSE: {:.2%}\n\nDiff in RMSE: {:.0%}'.format(
    gbst.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# pd.DataFrame({'Id':test['Id'],'SalePrice': np.exp(gbst.predict(test.drop(columns='Id')))}
#             ).to_csv('gbst {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)


The model score is 98.51%

Train RMSE: 4.91%

Test RMSE: 10.32%

Diff in RMSE: 5%


## XGBoost

In [24]:
# # read in data
# dtrain = xgb.DMatrix(x_train)
# dtest = xgb.DMatrix(x_test)
# # specify parameters via map
# param = {'max_depth':2, 'eta':1,'booster':'gbtree', 'objective':'reg:linear' }
# num_round = 2
# bst = xgb.train(param, dtrain, num_round)
# # make prediction
# preds = bst.predict(dtest)

# trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=bst.predict(x_train)))
# testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=bst.predict(x_test)))

# print('\nThe model score is {:.2%}\n\nTrain RMSE: {:.2%}\n\nTest RMSE: {:.2%}\n\nDiff in RMSE: {:.0%}'.format(
#         bst.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# # pd.DataFrame({'Id':test['Id'],'SalePrice': np.exp(xgb_model.predict(test.drop(columns='Id')))}
# #             ).to_csv('xgb_model {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
# #                                            ), index=False)

## Output csv

In [25]:
pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(ridcv.predict(test.drop(columns='Id')))}
            ).to_csv('ridcv {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
                                           ), index=False)
pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(enet.predict(test.drop(columns='Id')))}
            ).to_csv('enet {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
                                           ), index=False)
pd.DataFrame({'Id':test['Id'],'SalePrice': np.exp(lassocv.predict(test.drop(columns='Id')))}
            ).to_csv('lasso {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
                                           ), index=False)
pd.DataFrame({'Id':test['Id'],'SalePrice': np.exp(rf.predict(test.drop(columns='Id')))}
            ).to_csv('rf {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
                                           ), index=False)
pd.DataFrame({'Id':test['Id'],'SalePrice': np.exp(gbst.predict(test.drop(columns='Id')))}
            ).to_csv('gbst {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
                                           ), index=False)